## 基于单树GP的符号回归（Symbolic Regression）

基于单树GP的符号回归是指使用遗传编程（GP）生成数学公式来逼近一组数据的关系。


### Creator类
creator是一个工具类，其主要作用是创建新的类。在遗传编程中，通常需要自定义个体（Individual）和适应度（Fitness）类，因为不同的问题可能需要不同的适应度类型和个体结构。在DEAP中，我们可以使用creator来动态地创建这些类。

在下面的例子中，我们创建了一个最基本的单目标单树GP，可以使用base.Fitness和gp.PrimitiveTree来定义。

In [13]:
import math
import operator

from deap import base, creator, tools, gp


# 符号回归
def evalSymbReg(individual, pset):
    # 编译GP树为函数
    func = gp.compile(expr=individual, pset=pset)
    # 计算均方误差（Mean Square Error，MSE）
    mse = ((func(x) - x**2)**2 for x in range(-10, 10))
    return (math.fsum(mse),)

# 创建个体和适应度函数
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

### Register类
Register的作用类似于一个调度中心，它负责“注册”各种操作和函数。在遗传编程中，这些操作通常包括交叉（crossover）、变异（mutation）、选择（selection）和评估（evaluation）。通过register，我们可以将这些操作和相关的函数绑定在一起，以供后续算法使用。

In [14]:
import random

# 定义函数集合和终端集合
pset = gp.PrimitiveSet("MAIN", arity=1)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(operator.neg, 1)
pset.addEphemeralConstant("rand101", lambda: random.randint(-1, 1))
pset.renameArguments(ARG0='x')

# 定义遗传编程操作
toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)
toolbox.register("evaluate", evalSymbReg, pset=pset)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr, pset=pset)

### Algorithms类
Algorithms模块提供了一些现成的遗传算法和遗传编程的实现。例如，eaSimple是一个简单的遗传算法，它可以处理基本的选择、交叉、变异和演化迭代。

In [15]:
import numpy
from deap import algorithms

# 定义统计指标
stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
stats_size = tools.Statistics(len)
mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
mstats.register("avg", numpy.mean)
mstats.register("std", numpy.std)
mstats.register("min", numpy.min)
mstats.register("max", numpy.max)

# 使用默认算法
population = toolbox.population(n=300)
hof = tools.HallOfFame(1)
pop, log  = algorithms.eaSimple(population=population,
                           toolbox=toolbox, cxpb=0.5, mutpb=0.2, ngen=50, stats=mstats, halloffame=hof, verbose=True)


   	      	                    fitness                    	                      size                     
   	      	-----------------------------------------------	-----------------------------------------------
gen	nevals	avg    	gen	max   	min	nevals	std    	avg    	gen	max	min	nevals	std    
0  	300   	42078.7	0  	365994	0  	300   	31114.7	4.00667	0  	7  	2  	300   	1.68719
1  	160   	1.61412e+06	1  	2.41417e+08	0  	160   	1.91599e+07	4      	1  	12 	2  	160   	1.9799 
2  	192   	45381.3    	2  	3.19748e+06	0  	192   	185991     	4.12667	2  	12 	2  	192   	2.01096
3  	167   	98778.3    	3  	3.51947e+06	0  	167   	463075     	4.27   	3  	11 	2  	167   	1.97242
4  	176   	86987.9    	4  	3.19748e+06	0  	176   	440908     	4.47333	4  	13 	2  	176   	2.12821
5  	182   	89068.8    	5  	1.22679e+07	0  	182   	770251     	4.41   	5  	15 	2  	182   	2.21553
6  	171   	159554     	6  	1.33932e+07	0  	171   	1.10356e+06	4      	6  	11 	2  	171   	1.66333
7  	178   	104785     	7  	3.19748e+

由于DEAP重载了字符串运算符，因此可以直接输出结果。

In [16]:
print(str(hof[0]))

mul(x, x)
